In [1]:
import glob
import shutil
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from os import listdir, makedirs
from os.path import isfile, join, exists
%matplotlib inline
from tqdm import tqdm
import sys, multiprocessing, csv
from urllib import request, error
from io import BytesIO
import cv2
from sklearn import svm

In [2]:
train_wd = os.path.join(os.getcwd(),'train')
train_files = [f for f in listdir(train_wd) if isfile(join(train_wd, f))]
len(train_files)

88777

In [9]:
train_files

['00010a9d72c54031.jpg',
 '000110fc8d56bd9f.jpg',
 '00012fae9f7fec26.jpg',
 '0002c616173d4a4f.jpg',
 '00034c613d23375e.jpg',
 '00043bd3f1c2a31c.jpg',
 '00050e04290a568f.jpg',
 '00053b7a06cb7439.jpg',
 '0005c0eb461ba0f8.jpg',
 '00089dbd127821ea.jpg',
 '00096def60420f0d.jpg',
 '00098f7e3361f1fa.jpg',
 '0009ad6e1547c743.jpg',
 '0009bc176fb9a9c1.jpg',
 '000c5a6d7ff65f68.jpg',
 '000caa75ea83cea9.jpg',
 '000ce7f461f9cb05.jpg',
 '000e5a7773ba646f.jpg',
 '000edceb2acb414b.jpg',
 '000f284b69c70cbd.jpg',
 '000fd06733e5dd86.jpg',
 '000fe01d9eaf5d38.jpg',
 '001014d810a9101b.jpg',
 '0011b3587d8674a2.jpg',
 '0011ef38bae8e154.jpg',
 '001424dc4fd0482e.jpg',
 '00155b0fa4562a28.jpg',
 '0015682f56050acd.jpg',
 '0015fe12f2889106.jpg',
 '0016a82c2743c9e3.jpg',
 '0017be7c91173d0e.jpg',
 '00180e533f76ae70.jpg',
 '00195ec37ce05bcf.jpg',
 '0019f56bf53d2e5d.jpg',
 '001a8d320f595b98.jpg',
 '001b597c7ed6a439.jpg',
 '001c2fe76e7d1366.jpg',
 '001cb4a7954bd0ae.jpg',
 '001cf7c30c2645c1.jpg',
 '001e99c25a1c9398.jpg',


In [5]:
df = pd.read_csv("train.csv")
df1 = pd.read_csv("test.csv")
df2 = pd.read_csv("test1.csv")

In [13]:
df["id"]

0          cacf8152e2d2ae60
1          0a58358a2afd3e4e
2          6b2bb500b6a38aa0
3          b399f09dee9c3c67
4          19ace29d77a5be66
5          2c9c54b62f0a6a37
6          0aac70a1de44ced6
7          de770bc720f8e714
8          dc9457d703e612ad
9          3060f5f75d936abb
10         12f403e97cb69f39
11         fee38235bfae7e99
12         e53f66e7bab7cef2
13         0a8339ce84ee6592
14         823355e68aa327d1
15         86608c54eced1b99
16         3cbab75ec1e879b8
17         ab91c5915f26b679
18         4a699578529b45ce
19         b9260b9875a315b1
20         88d3c782afad61ab
21         42ca212aeebe50f3
22         c9a657ed15fedddb
23         270140b07481fb7e
24         493ee4b01a3deb82
25         7431754cb2c13035
26         17f3719ad574e36c
27         7a139d419a3a009f
28         d21e1c3455134c0d
29         5ca6dc031bd3f2d9
                 ...       
1224999    78fc2ce2eee977e8
1225000    33646e9c3e951098
1225001    b7ca718adedccd6c
1225002    d34c94c9f36fafae
1225003    f6bc33a59

In [3]:
import pickle

train_labels = pickle.load(open("trainlabels.pickle", "rb"))

In [4]:
len(train_labels)

88775

In [4]:
test_wd = os.path.join(os.getcwd(),'test')
test_files = [f for f in listdir(test_wd) if isfile(join(test_wd, f))]
len(test_files)

115502

In [8]:
len(set(df["landmark_id"]))

14951

In [3]:
import csv
all_train = {}
iter = 0

with open(os.path.join(os.getcwd(), 'train.csv')) as train_csvfile:
    reader = csv.DictReader(train_csvfile)
    # create lookup dictionary for images that have been successfully downloaded
    for row in reader:
        all_train[row['id']] = row['landmark_id']

In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNN

# load training data

train_labels = []
train_vectors = []
bad_train_data = []

test_vectors = []

itr = 0
img_size = (32,32)
for filename in tqdm(train_files):
#     img = cv2.imread(os.path.join(test_wd,filename),0) # 0 second input for greyscale.
#     sift=cv2.ORB_create()
#     kp, des=sift.detectAndCompute(img,None)
#     try:
#         test_vectors.append(des.flatten())
#     except:
#         continue
#     # find targets
    fn = filename.replace('.jpg','')
    train_labels.append(all_train[fn])

100%|███████████████████████████████████████████████████████████████████████| 88777/88777 [00:00<00:00, 1292527.30it/s]


In [18]:
# import pickle

# pickle.dump(test_vectors, open("test.p", "wb"))
import pickle

# train_labels = pickle.load(open("trainlabels.pickle", "rb"))
train_vectors = pickle.load(open("trainvectors.pickle", "rb"))
test_vectors = pickle.load(open("test.p", "rb"))

In [20]:
to_list = [len(a) for a in test_vectors]

In [25]:
len(train_vectors[0])/32

393.0

In [13]:
train_vectors = pickle.load(open("trainvectors.pickle", "rb"))

,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...
2,0001bbb682d45002,https://lh3.googleusercontent.com/-kloLenz1xZk...
3,0002362830cfe3a3,https://lh3.googleusercontent.com/-N6z79jNZYTg...
4,000270c9100de789,https://lh3.googleusercontent.com/-keriHaVOq1U...
5,0002b0fab5d3ccc4,https://lh3.googleusercontent.com/-ciWklpsrab8...
6,000396be3c24830a,https://lh3.googleusercontent.com/-6W9F179t59Q...
7,000506dc6ab3a40e,https://lh3.googleusercontent.com/-_XHsAXB2LZA...
8,0005292fc4b005a3,https://lh3.googleusercontent.com/-RBZ4F1ZKNc0...
9,0005456a82264bc8,https://lh3.googleusercontent.com/-MRK7_uiKO6A...


In [28]:
import torch 


'0.3.1.post2'